## Dependencies

In [1]:
import glob
import warnings
from tensorflow_hub import KerasLayer
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Concatenate
from googleqa_utilityscript import *
from googleqa_map_utilityscript import *
import bert_tokenization as tokenization


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
BERT_PATH = '/kaggle/input/tf-hub-bert-base/bert_base_uncased'
VOCAB_PATH = BERT_PATH + '/assets/vocab.txt'
model_path_list = glob.glob('/kaggle/input/135bert-base-best/' + '*.h5')
model_path_list.sort()
print('Models to predict:')
print(*model_path_list, sep = "\n")

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Models to predict:
/kaggle/input/135bert-base-best/135-BERT_base_uncased_model_fold_1.h5
/kaggle/input/135bert-base-best/135-BERT_base_uncased_model_fold_2.h5
/kaggle/input/135bert-base-best/135-BERT_base_uncased_model_fold_3.h5
/kaggle/input/135bert-base-best/135-BERT_base_uncased_model_fold_4.h5
/kaggle/input/135bert-base-best/135-BERT_base_uncased_model_fold_5.h5
Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']

# for feature in text_features:
#     # Lower
#     test[feature] = test[feature].apply(lambda x: x.lower())
#     # Map misspellings
#     test[feature] = test[feature].apply(lambda x: map_misspellings(x))
#     # Map contractions
#     test[feature] = test[feature].apply(lambda x: map_contraction(x))
#     # Trim text
#     test[feature] = test[feature].apply(lambda x: x.strip())

# Model parameters

In [5]:
N_CLASS = len(target_cols)
MAX_SEQUENCE_LENGTH = 512

## Test set

In [6]:
tokenizer = tokenization.FullTokenizer(VOCAB_PATH, do_lower_case=True)

# Test features
X_test = compute_input_arays(test, text_features, tokenizer, MAX_SEQUENCE_LENGTH)

# Model

In [7]:
def model_fn():
    input_word_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids')
    input_masks = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks')
    segment_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids')

    bert_layer = KerasLayer(BERT_PATH, trainable=False)
    pooled_output, sequence_output = bert_layer([input_word_ids, input_masks, segment_ids])

    x = GlobalAveragePooling1D()(sequence_output)
    x = Dropout(0.2)(x)
    output = Dense(N_CLASS, activation="sigmoid", name="output")(x)

    model = Model(inputs=[input_word_ids, input_masks, segment_ids], outputs=output)
    
    return model

# Make predictions

In [8]:
Y_test = np.zeros((len(test), N_CLASS))
weights = [.3, .2, .1, .1, .3]

for index, model_path in enumerate(model_path_list):
    model = model_fn()
    model.load_weights(model_path)
    Y_test += model.predict(X_test) * weights[index]

In [9]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.930913,0.609927,0.251926,0.507946,0.543561,0.537395,0.658376,0.652535,0.616179,...,0.889106,0.893015,0.533351,0.949722,0.944191,0.763157,0.080299,0.060476,0.766798,0.910338
1,46,0.880295,0.485868,0.003778,0.820680,0.792189,0.935278,0.579212,0.492772,0.034706,...,0.692854,0.949465,0.673481,0.969196,0.981332,0.873790,0.867262,0.105496,0.116582,0.890491
2,70,0.922950,0.649068,0.026705,0.798220,0.855879,0.927121,0.601661,0.538697,0.166183,...,0.839834,0.944391,0.617352,0.973009,0.976235,0.867414,0.118434,0.069570,0.841496,0.914951
3,132,0.891170,0.428924,0.003649,0.754020,0.732401,0.907442,0.574072,0.433344,0.094059,...,0.717058,0.954256,0.663465,0.973860,0.986495,0.902874,0.813498,0.147134,0.443378,0.917913
4,200,0.916288,0.459815,0.038294,0.826846,0.684300,0.811068,0.646124,0.617848,0.072097,...,0.714831,0.920054,0.635694,0.955751,0.960656,0.827351,0.142248,0.095775,0.636152,0.902983


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.894293,0.586029,0.034512,0.748126,0.781674,0.846270,0.594644,0.501698,0.204336,...,0.800681,0.939378,0.662053,0.964614,0.974349,0.869616,0.490703,0.126088,0.535749,0.913126
std,2812.670060,0.043793,0.120968,0.064253,0.096637,0.106063,0.128862,0.047468,0.085335,0.166062,...,0.079932,0.020524,0.048175,0.011704,0.011256,0.036499,0.305613,0.061576,0.258139,0.018590
min,39.000000,0.765843,0.364862,0.001424,0.298113,0.339501,0.266506,0.503801,0.364843,0.004467,...,0.574068,0.863958,0.533351,0.915030,0.910175,0.751024,0.006728,0.003901,0.048548,0.842399
25%,2572.000000,0.865695,0.479086,0.005962,0.695175,0.734180,0.824658,0.556447,0.435763,0.077197,...,0.739646,0.927859,0.625763,0.957820,0.969093,0.846487,0.167912,0.082168,0.320771,0.900239
50%,5093.000000,0.896355,0.585595,0.010140,0.757082,0.793487,0.894092,0.585409,0.476182,0.144756,...,0.805269,0.942794,0.658953,0.966068,0.976580,0.874258,0.535723,0.125563,0.527289,0.913884
75%,7482.000000,0.928922,0.676441,0.027690,0.809428,0.855248,0.925070,0.622288,0.551016,0.297346,...,0.867536,0.954343,0.693908,0.972938,0.982295,0.895323,0.766708,0.169933,0.751716,0.927152
max,9640.000000,0.979504,0.913388,0.585212,0.958399,0.972254,0.977970,0.743187,0.766895,0.761628,...,0.966300,0.982928,0.818941,0.989924,0.992910,0.963898,0.953693,0.330352,0.987337,0.957961
